### Problem formulation

some text here
<br/><br/><br/>

In [12]:
import findspark
findspark.init()

### Import all needed libs

In [3]:
# basically spark
import pyspark
from pyspark import SparkContext
from pyspark.sql import SparkSession

# processing
import re
from datetime import datetime

# text preprocessing
import nltk
from nltk.corpus import stopwords

### Global variables definition

**User-specific variables**  
Please feel free to tweak those variables as you wish. For example, you can set number of last hours to get hottest topics.

In [5]:
number_of_hours_to_get_topics = 2
frame_start_datetime = "Sun Jun 30 00:00:00 +0000 2019"
frame_finish_datetime = "Sun Jun 30 23:00:00 +0000 2019"

**Technical variables**  
Those variables are needed to connect to db and other technical stuff.

In [ ]:
tweets_table = "usa_training_tweets_30_06.training_tweets_collection"

### Handy functions

In [6]:
### we should put this block in the util functions file

# def str_to_tweeter_datetime():
#     ts2 = datetime.strptime(frame_start_datetime,'%a %b %d %H:%M:%S %z %Y')
#     ts = time.strftime('%Y-%m-%d %H:%M:%S', time.strptime(frame_start_datetime,'%a %b %d %H:%M:%S +0000 %Y'))

In [7]:
def str_tweet_to_datetime(frame_datetime):
    ts = datetime.strptime(frame_datetime,'%a %b %d %H:%M:%S %z %Y')
    return ts

def datetime_to_tweet_str(frame_datetime):
    #print(type(frame_datetime))
    ts = datetime.strftime(frame_datetime, '%a %b %d %H:%M:%S %z %Y')
    return ts

In [8]:
dtf = str_tweet_to_datetime(frame_finish_datetime)

dtf

datetime.datetime(2019, 6, 30, 23, 0, tzinfo=datetime.timezone.utc)

In [9]:
datetime_to_tweet_str(dtf)

'Sun Jun 30 23:00:00 +0000 2019'

### Reading CSV

In [14]:
df = spark.read.load('./get-tweets-by-geolocation/data/usa_training_tweets_30_06.csv')

Py4JJavaError: An error occurred while calling o132.load.
: org.apache.spark.SparkException: Job aborted due to stage failure: Task 0 in stage 0.0 failed 1 times, most recent failure: Lost task 0.0 in stage 0.0 (TID 0, localhost, executor driver): org.apache.spark.SparkException: Exception thrown in awaitResult: 
	at org.apache.spark.util.ThreadUtils$.awaitResult(ThreadUtils.scala:226)
	at org.apache.spark.util.ThreadUtils$.parmap(ThreadUtils.scala:290)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$.readParquetFootersInParallel(ParquetFileFormat.scala:538)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:611)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetFileFormat$$anonfun$9.apply(ParquetFileFormat.scala:603)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(RDD.scala:801)
	at org.apache.spark.rdd.RDD$$anonfun$mapPartitions$1$$anonfun$apply$23.apply(R

### Connecting to MongoDB

**Creating Spark connector**

In [10]:
spark = SparkSession.builder.appName("pipeline") \
    .config('spark.mongodb.input.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.mongodb.output.uri', 'mongodb://localhost:27017/'+tweets_table) \
    .config('spark.jars.packages', 'org.mongodb.spark:mongo-spark-connector_2.11:2.3.1') \
    .config('spark.mongodb.input.partitioner', 'MongoPaginateBySizePartitioner') \
    .getOrCreate()

## Huge bottleneck, isn't it?

**Creating Spark dataframe on top of DB**

In [69]:
df = spark.read.format("com.mongodb.spark.sql.DefaultSource").load()

In [70]:
# df.createOrReplaceTempView("temp")

In [71]:
df.printSchema()

root
 |-- _id: struct (nullable = true)
 |    |-- oid: string (nullable = true)
 |-- contributors: null (nullable = true)
 |-- coordinates: struct (nullable = true)
 |    |-- type: string (nullable = true)
 |    |-- coordinates: array (nullable = true)
 |    |    |-- element: double (containsNull = true)
 |-- created_at: string (nullable = true)
 |-- display_text_range: array (nullable = true)
 |    |-- element: integer (containsNull = true)
 |-- entities: struct (nullable = true)
 |    |-- hashtags: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- text: string (nullable = true)
 |    |    |    |-- indices: array (nullable = true)
 |    |    |    |    |-- element: integer (containsNull = true)
 |    |-- media: array (nullable = true)
 |    |    |-- element: struct (containsNull = true)
 |    |    |    |-- additional_media_info: struct (nullable = true)
 |    |    |    |    |-- monetizable: boolean (nullable = true)
 |    |    |    |-- dis

In [57]:
# Sample query
# sample = spark.sql('SELECT * FROM temp LIMIT 10')
# sample.collect()

# Data description (Женя)
how e collected it?

for what timeframe we have a date and for what region (NY/USA)<br/>
size of tweets that we have (for NY/USA)

### Data preprocessing

#### Selecting data from {{ frame_start_datetime }} to {{frame_finish_datetime}}

In [11]:
sample = spark.sql('SELECT * FROM root LIMIT 10')

AnalysisException: 'Table or view not found: root; line 1 pos 14'

In [59]:
StopWords = stopwords.words("english")

tokens = df.select('text').rdd \
    .map(lambda x: x[0]) \
    .map( lambda document: re.split(" ", document))          \
    .map( lambda word: [x for x in word if x.isalpha()])           \
    .map( lambda word: [x for x in word if len(x) > 3] )           \
    .map( lambda word: [x for x in word if x not in StopWords])    \


AnalysisException: "cannot resolve '`text`' given input columns: [];;\n'Project ['text]\n+- Relation[] MongoRelation(MongoRDD[14] at RDD at MongoRDD.scala:51,Some(StructType()))\n"

In [60]:
tokens.collect()

NameError: name 'tokens' is not defined

### Topic modeling (LDA)

### Hot topics in the USA from [Google trends](https://trends.google.com/trends/explore?geo=US)

In [85]:
sc = pyspark.SparkContext.getOrCreate()
spark = SparkSession(sc)

Set window time for interesting

In [117]:
window_time = (datetime(2019, 6, 5, 0, 0), datetime(2019, 6, 5, 0, 0))

Ranked by Top

In [115]:
df = spark.read.csv("data/google-trends/google-trends-search-queries-by-top-US.csv", inferSchema=True, header=True)

In [116]:
search_queries = df.filter((df.Date >= window_time[0]) & (df.Date <= window_time[1]))
search_queries.select('Search queries').show()

+--------------+
|Search queries|
+--------------+
|       weather|
|      facebook|
|       youtube|
|        google|
|        amazon|
|          news|
|    craigslist|
|       walmart|
|         pizza|
|        movies|
|virginia beach|
|           nba|
|            ma|
|     instagram|
|     champions|
|         yahoo|
|    home depot|
|         apple|
|        people|
|       netflix|
+--------------+
only showing top 20 rows



In [113]:
df = spark.read.csv("data/google-trends/google-trends-search-topics-by-top-US.csv", inferSchema=True, header=True)
search_topics = df.filter((df.Date >= window_time[0]) & (df.Date <= window_time[1]))
cols = ("Search topics", "Top")
search_topics.select(*cols).show()

+--------------------+---+
|       Search topics|Top|
+--------------------+---+
|     Weather - Topic|100|
|        2019 - Topic| 98|
|Google Search - T...| 85|
|Google - Technolo...| 85|
|Facebook - Social...| 73|
|Facebook, Inc. - ...| 71|
|YouTube - Video s...| 68|
|        Film - Topic| 53|
|    Texas - US State| 50|
|Amazon.com - E-co...| 48|
|Car - Transportat...| 47|
|       Sales - Topic| 43|
|California - US S...| 42|
|  Definition - Topic| 40|
|.com - Top level ...| 38|
|        City - Topic| 38|
|United States - R...| 37|
|  Florida - US State| 37|
|Craigslist, Inc. ...| 37|
|         Job - Topic| 34|
+--------------------+---+



Ranked by rising

In [98]:
df = spark.read.csv("data/google-trends/google-trends-search-queries-by-rising-US.csv", inferSchema=True, header=True)
search_queries_by_rising = df.filter((df.Date >= window_time[0]) & (df.Date <= window_time[1])).collect()
search_queries_by_rising

[Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='fair oaks farm', Rising='+1,150%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='pokemon sword and shield', Rising='+700%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='tracy morgan', Rising='+450%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='black mirror', Rising='+400%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='mega millions drawing', Rising='+300%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='mega millions numbers', Rising='+300%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='chemical castration', Rising='+300%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='mega millions', Rising='+300%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='measure ee', Rising='+200%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search queries='portugal fc', Rising='+200%'),
 Row(Date=d

In [101]:
df = spark.read.csv("data/google-trends/google-trends-search-topics-by-rising-US.csv", inferSchema=True, header=True)
search_topics_by_rising = df.filter((df.Date >= window_time[0]) & (df.Date <= window_time[1])).collect()
search_topics_by_rising

[Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search topics='Mega Millions - Topic', Rising='+250%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search topics='Pokémon - Topic', Rising='+100%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search topics='Radar - Topic', Rising='+90%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search topics='Golden State Warriors - Basketball team', Rising='+40%'),
 Row(Date=datetime.datetime(2019, 6, 5, 0, 0), Search topics='Weather - Topic', Rising='+40%')]

#### Hot topics - google trends (directly) (probably this will be removed)

In [118]:
!ls

data			   README.md
get-google-trends-data	   realtime_twitter_topic_modeling_pyspark.ipynb
get-tweets-by-geolocation  sample_lda_data.txt
LICENSE			   spark_LDA.ipynb
python-data-preprocessing  spark_LDA_v2.ipynb
python_LDA.ipynb	   spark-warehouse


In [128]:
from get_google_trends_data.pytrends.pytrends.request import TrendReq
pytrend = TrendReq()
timeframe = "2019-06-5 2019-06-20"
pytrend.build_payload(kw_list=[' '], geo='US', timeframe=timeframe)
topics_df = pytrend.related_top_topics_by_top()

In [129]:
topics_df

,Search topics,Top
0,2019 - Topic,100
1,Weather - Topic,85
2,Facebook - Social networking service,73
3,"Facebook, Inc. - Social network company",73
4,Google Search - Topic,69
5,Google - Technology company,67
6,YouTube - Video sharing company,64
7,Film - Topic,60
8,Car - Transportation mode,47
9,Texas - US State,46


### Conclusion